<a href="https://colab.research.google.com/github/Shadabur-Rahaman/30-days-ml-projects/blob/main/Day_29_ml_webapp_image_classification/notebook/Day_29_ml_webapp_image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML Model Deployment with Gradio
## Day 29/30 of Machine Learning Project

**Project Goal**: Create a web interface to demo your trained ML model using Gradio
**Key Features**:
- Simple UI for model predictions
- Support for text/image/audio input
- Shareable public link
- Customizable interface

## 1. Install and Import Required Libraries

In [12]:
import gradio as gr
import torch
import torchvision.transforms as transforms
from PIL import Image
import requests
import os
from datetime import datetime
import csv

## 2. Load Your Trained Model

In [13]:
# Create output directories
os.makedirs("saved_images", exist_ok=True)
os.makedirs("predictions", exist_ok=True)

# 1. Load pre-trained ResNet model
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
model.eval()

# 2. Download ImageNet class labels
response = requests.get("https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt")
labels = response.text.split("\n")

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## 3. Define Prediction Functions

In [15]:
# 3. Preprocessing function
def preprocess_image(image):
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ])
    return transform(image).unsqueeze(0)

# 4. Prediction and saving function
def classify_and_save(img):
    # Handle different input types
    if isinstance(img, str):  # File path
        img_pil = Image.open(img).convert('RGB')
    elif isinstance(img, np.ndarray):  # Numpy array
        img_pil = Image.fromarray(img.astype('uint8'), 'RGB')
    else:  # Gradio temp file
        img_pil = Image.open(img.name).convert('RGB')

    # Create timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    # Save original image
    orig_img_path = f"saved_images/original_{timestamp}.jpg"
    img_pil.save(orig_img_path)

    # Preprocess and predict
    input_tensor = preprocess_image(img_pil)
    with torch.no_grad():
        output = model(input_tensor)

    # Get probabilities
    probabilities = torch.nn.functional.softmax(output[0], dim=0)
    top5_prob, top5_catid = torch.topk(probabilities, 5)

    # Prepare results
    results = {
        labels[top5_catid[i]]: float(top5_prob[i])
        for i in range(top5_prob.size(0))
    }

    # Save predictions to CSV
    csv_path = f"predictions/predictions_{timestamp}.csv"
    with open(csv_path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Class', 'Confidence'])
        for class_name, confidence in results.items():
            writer.writerow([class_name, f"{confidence:.4f}"])

    # Create annotated image
    img_annotated = img_pil.copy()
    return results, orig_img_path, csv_path

## 4. Create Gradio Interface

In [16]:
# 5. Create Gradio interface with full-screen option
with gr.Blocks(theme=gr.themes.Soft(), css=".gradio-container {max-width: 1200px !important}") as app:
    gr.Markdown("# 🖼️ Image Classification App")
    gr.Markdown("Upload an image to classify and save results")

    with gr.Row():
        with gr.Column(scale=1):
            image_input = gr.Image(type="filepath", label="Input Image")
            submit_btn = gr.Button("Classify & Save", variant="primary")

            with gr.Accordion("Output Files", open=False):
                saved_image = gr.Image(label="Saved Image", interactive=False)
                csv_display = gr.File(label="Download Predictions")

        with gr.Column(scale=2):
            label_output = gr.Label(label="Top Predictions", num_top_classes=5)
            plot_output = gr.BarPlot(
                x="class",
                y="confidence",
                title="Prediction Confidence",
                vertical=False,
                height=300
            )

    # Examples with full-screen option
    examples = gr.Examples(
        examples=[
            ["https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba"],
            ["https://images.unsplash.com/photo-1543852786-1cf6624b9987"],
            ["https://images.unsplash.com/photo-1583512603805-3cc6b41f3edb"]
        ],
        inputs=[image_input],
        label="Example Images (click then press Classify)",
        examples_per_page=3
    )

    # Full-screen button
    fullscreen_btn = gr.Button("Open Full Screen", link="/?__theme=dark")

    # Processing function
    def process_image(image):
        results, img_path, csv_path = classify_and_save(image)

        # Prepare plot data
        sorted_results = sorted(results.items(), key=lambda x: x[1], reverse=True)
        plot_data = pd.DataFrame({
            "class": [k.split(",")[0] for k, v in sorted_results],
            "confidence": [v for k, v in sorted_results]
        })

        return results, plot_data, img_path, csv_path

    # Event handler
    submit_btn.click(
        fn=process_image,
        inputs=image_input,
        outputs=[label_output, plot_output, saved_image, csv_display]
    )

## 5. Launch the Web App

In [17]:
# 7. Launch the app
if __name__ == "__main__":
    app.launch()

# Launch blocks demo (uncomment to run)
# demo_app.launch()

# Note: In notebook environments, use:
# demo_app.launch(share=True, inbrowser=True)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a28fa799b9091e8acb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## 6. Deployment Options

**Permanent Deployment Options**:
- Hugging Face Spaces (free)
- AWS/Azure/GCP cloud services
- Docker containerization
- Gradio sharing (temporary 72-hour link)

[![Hugging Face Spaces](https://img.shields.io/badge/%F0%9F%A4%97%20Hugging%20Face-Spaces-blue)](https://huggingface.co/spaces)

## 7. Create Requirements File for Deployment

In [ ]:
%%writefile requirements.txt
torch
transformers
gradio
numpy
Pillow
matplotlib